In [1]:
import pandas as pd 
import numpy as np

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
# for returning lat long coordinates from addresses
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

df = pd.read_csv('jersey_city_residential.csv')

In [2]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307"
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307"
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307"
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307"
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307"
...,...,...,...,...
41799,98 GARFIELD AVE.,"SWINNEY, BRENDA",98 GARFIELD AVE.,"JERSEY CITY, N.J. 07305"
41800,100 GARFIELD AVE.,"SERVIDA, AUREA",100 GARFIELD AVE.,"JERSEY CITY, NJ 07305"
41801,102 GARFIELD AVE.,"DIAZ, ANGEL & GALADITA",5405 STEVEHAVEN LN.,"CUMMING , GA. 30028"
41802,104 GARFIELD AVE.,"RODRIGUEZ, BENJAMIN",104 GARFIELD AVE.,"JERSEY CITY, N.J. 07305"


In [4]:
# number of times property appears in first column
def propertyAppearances(address=str): 
    return len(df[df.propertyLocation == address])

In [5]:
# number of times owner appears in second column
def propertiesOwned(owner=str):
    return len(df[df.ownersName == owner])

In [6]:
df['propertiesOwned'] = [propertiesOwned(owner) for owner in df.ownersName]
df['units'] = [propertyAppearances(address) for address in df.propertyLocation]

In [7]:
df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
df['propertyFullAddress'] = df.propertyLocation + ' Jersey City, NJ'

In [8]:
df.columns

Index(['propertyLocation', 'ownersName', 'ownersMailingAddress',
       'cityStateZip', 'propertiesOwned', 'units', 'ownersFullMailingAddress',
       'propertyFullAddress'],
      dtype='object')

In [9]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,propertiesOwned,units,ownersFullMailingAddress,propertyFullAddress
0,677 LIBERTY AVE.,"PEDDI, PRADEEP",677 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1,1,"677 LIBERTY AVE., JERSEY CITY, N.J. 07307","677 LIBERTY AVE. Jersey City, NJ"
1,675 LIBERTY AVE.,"PAREJA, HENRY A. & MIRYAM C.",675 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1,1,"675 LIBERTY AVE., JERSEY CITY, N.J. 07307","675 LIBERTY AVE. Jersey City, NJ"
2,673 LIBERTY AVE.,"HIRPARA, PRAVIN",673 LIBERTY AVE.,"JERSEY CITY, NJ 07307",1,1,"673 LIBERTY AVE., JERSEY CITY, NJ 07307","673 LIBERTY AVE. Jersey City, NJ"
3,671 LIBERTY AVE.,"SINGH, HONEY",671 LIBERTY AVE.,"JERSEY CITY, NJ 07307",1,1,"671 LIBERTY AVE., JERSEY CITY, NJ 07307","671 LIBERTY AVE. Jersey City, NJ"
4,669 LIBERTY AVE.,"MARTIN, BENIGNO & CARIDAD",669 LIBERTY AVE.,"JERSEY CITY, N.J. 07307",1,1,"669 LIBERTY AVE., JERSEY CITY, N.J. 07307","669 LIBERTY AVE. Jersey City, NJ"
...,...,...,...,...,...,...,...,...
41799,98 GARFIELD AVE.,"SWINNEY, BRENDA",98 GARFIELD AVE.,"JERSEY CITY, N.J. 07305",2,1,"98 GARFIELD AVE., JERSEY CITY, N.J. 07305","98 GARFIELD AVE. Jersey City, NJ"
41800,100 GARFIELD AVE.,"SERVIDA, AUREA",100 GARFIELD AVE.,"JERSEY CITY, NJ 07305",1,1,"100 GARFIELD AVE., JERSEY CITY, NJ 07305","100 GARFIELD AVE. Jersey City, NJ"
41801,102 GARFIELD AVE.,"DIAZ, ANGEL & GALADITA",5405 STEVEHAVEN LN.,"CUMMING , GA. 30028",1,1,"5405 STEVEHAVEN LN., CUMMING , GA. 30028","102 GARFIELD AVE. Jersey City, NJ"
41802,104 GARFIELD AVE.,"RODRIGUEZ, BENJAMIN",104 GARFIELD AVE.,"JERSEY CITY, N.J. 07305",1,1,"104 GARFIELD AVE., JERSEY CITY, N.J. 07305","104 GARFIELD AVE. Jersey City, NJ"


In [15]:
df[df.propertyLocation == '1 CONGRESS ST.']

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,propertiesOwned,units,ownersFullMailingAddress,propertyFullAddress
3433,1 CONGRESS ST.,KIMBERLY ARMS % HOBOKEN RLTY.MGNT,1018 WASHINGTON ST.3RD FL,"HOBOEKN, NJ 07030",1,32,"1018 WASHINGTON ST.3RD FL ,HOBOEKN, NJ 07030","1 CONGRESS ST. Jersey City, NJ"
3434,1 CONGRESS ST.,"HANNI, ROBERT & EVELYN",2709 CRESMONT AVE,"EASTON, PA 18045",1,32,"2709 CRESMONT AVE ,EASTON, PA 18045","1 CONGRESS ST. Jersey City, NJ"
3435,1 CONGRESS ST.,"MILLER, FRANKLIN T. & ALLEN R.","1 CONGRESS ST., # A-3","JERSEY CITY, NJ 07307",1,32,"1 CONGRESS ST., # A-3 ,JERSEY CITY, NJ 07307","1 CONGRESS ST. Jersey City, NJ"
3436,1 CONGRESS ST.,"JAMES, ANDREW & SCHUBERT",444 WASHINGTON BLVD #4506,"JERSEY CITY, NJ 07310",2,32,"444 WASHINGTON BLVD #4506 ,JERSEY CITY, NJ 07310","1 CONGRESS ST. Jersey City, NJ"
3437,1 CONGRESS ST.,"AGINA, OMAR E.",32 WOLF DRIVE,"WANAQUE, N.J. 07465",1,32,"32 WOLF DRIVE ,WANAQUE, N.J. 07465","1 CONGRESS ST. Jersey City, NJ"
3438,1 CONGRESS ST.,"WOOD, SARAH & MOORE, DOUGLAS",1 CONGRESS ST.#A6,"JERSEY CITY, N.J. 07307",1,32,"1 CONGRESS ST.#A6 ,JERSEY CITY, N.J. 07307","1 CONGRESS ST. Jersey City, NJ"
3439,1 CONGRESS ST.,"PATEL, DEVON & JASANI, NISHA",1 CONGRESS ST.#A7,"JERSEY CITY, N.J. 07307",1,32,"1 CONGRESS ST.#A7 ,JERSEY CITY, N.J. 07307","1 CONGRESS ST. Jersey City, NJ"
3440,1 CONGRESS ST.,"DIAZ, ABIGAIL","1 CONGRESS ST.,#A8","JERSEY CITY, NJ 07307",1,32,"1 CONGRESS ST.,#A8 ,JERSEY CITY, NJ 07307","1 CONGRESS ST. Jersey City, NJ"
3441,1 CONGRESS ST.,"EVANS, KAREN",1 CONGRESS ST.,"JERSEY CITY, NJ 07307",1,32,"1 CONGRESS ST. ,JERSEY CITY, NJ 07307","1 CONGRESS ST. Jersey City, NJ"
3442,1 CONGRESS ST.,"FRIO, DOMINIC",1321 WASHINGTON ST.,"HOBOKEN, NJ 07030",1,32,"1321 WASHINGTON ST. ,HOBOKEN, NJ 07030","1 CONGRESS ST. Jersey City, NJ"
